# Dual-Task

## DT | Configuration

In [2]:
# Configuration
from arcface_torch.configs.aihub_r50_onegpu import config as aihub_config
from arcface_torch.configs.base import config as cfg

In [3]:
cfg.update(aihub_config)
cfg.output = "work_dirs/aihub_r50_onegpu"

## DT | transforms

In [56]:
import torchvision

image_size = 112
aihub_mean = [0.5444, 0.4335, 0.3800]
aihub_std = [0.2672, 0.2295, 0.2156]

nia_train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.RandomApply(
            [
                torchvision.transforms.RandomAffine(degrees=10, shear=16),
                torchvision.transforms.RandomHorizontalFlip(p=1.0),
            ],
            p=0.5,
        ),
        torchvision.transforms.Resize((256, 256)),
        torchvision.transforms.RandomCrop((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)

nia_valid_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)

aihub_train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(image_size, image_size)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

aihub_valid_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(image_size, image_size)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

aihub_test_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(image_size, image_size)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

## DT | AIHub DataFrame

In [20]:
from fpt.path import DTFR
from fpt.data import join_face_df

face_df = join_face_df(DTFR, "aihub_family")

## DT | Dataset

In [71]:
import os
from easydict import EasyDict as edict
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from facenet.datasets.AIHubDataset import AIHubDataset
from nia_age.data import NiaDataset as nia

RANGE_TO_MEDIAN = nia.RANGE_TO_MEDIAN
AGE_GROUPS = nia.AGE_GROUPS
GROUP_TO_INDEX = {group: index for index, group in enumerate(AGE_GROUPS)}
age_to_age_groups = nia.age_to_age_groups


class FaceAgeDataset(Dataset):
    def __init__(self, root_dir, face_df, transform):
        super(FaceAgeDataset, self).__init__()
        self.face_dataset = ImageFolder(root=root_dir, transform=transform)
        self.face_df = face_df
        self.class_to_idx = self.face_dataset.class_to_idx
        self.samples = self.face_dataset.samples  # 이 줄을 추가합니다.


    def __len__(self):
        return len(self.face_dataset)

    def __getitem__(self, index):
        image, face_label = self.face_dataset[index]
        path, _ = self.face_dataset.samples[index]
        *_, key = os.path.splitext(path)[0].split("/")
        row = face_df.loc[key]
        sample = edict(
            {
                "image": image,
                "age": row.age,
                "age_class": GROUP_TO_INDEX[row.age_group],
                "file": path,
                "data_type": row.category,
                "family_id": row.family_id,
                "personal_id": row.target,
                "face_label": face_label,
                "key": key,
            }
        )
        return sample

In [72]:
face_age_train_dataset = FaceAgeDataset(
    root_dir="/home/jupyter/data/face-image/train_aihub_family",
    face_df=face_df,
    transform=aihub_train_transforms,
)

face_age_valid_dataset = FaceAgeDataset(
    root_dir="/home/jupyter/data/face-image/valid_aihub_family",
    face_df=face_df,
    transform=aihub_valid_transforms,
)

face_age_test_dataset = FaceAgeDataset(
    root_dir="/home/jupyter/data/face-image/test_aihub_family",
    face_df=face_df,
    transform=aihub_test_transforms,
)

aihub_pairs_valid_dataset = AIHubDataset(
    dir="/home/jupyter/data/face-image/valid_aihub_family",
    pairs_path="/home/jupyter/data/pairs/valid/pairs_Age.txt",
    transform=aihub_valid_transforms,
)

aihub_pairs_test_dataset = AIHubDataset(
    dir="/home/jupyter/data/face-image/test_aihub_family",
    pairs_path="/home/jupyter/data/pairs/test/pairs_Age.txt",
    transform=aihub_valid_transforms,
)

## DT | DataLoader

In [60]:
from torch.utils.data import DataLoader

train_batch_size = 16
valid_batch_size = 1
test_batch_size = 1

face_age_train_loader = DataLoader(
    face_age_train_dataset,
    batch_size=train_batch_size,
    num_workers=0,
    shuffle=True,
)

face_age_valid_loader = DataLoader(
    face_age_valid_dataset,
    batch_size=valid_batch_size,
    num_workers=0,
    shuffle=False,
)

face_age_test_loader = DataLoader(
    face_age_test_dataset,
    batch_size=test_batch_size,
    num_workers=0,
    shuffle=False,
)

aihub_pairs_valid_loader = DataLoader(
    aihub_pairs_valid_dataset,
    batch_size=valid_batch_size,
    num_workers=0,
    shuffle=False,
)

aihub_pairs_test_loader = DataLoader(
    aihub_pairs_test_dataset,
    batch_size=test_batch_size,
    num_workers=0,
    shuffle=False,
)

## DT | Loss function

In [125]:
from typing import Callable
import torch
from torch.nn import CrossEntropyLoss
from torch.nn.functional import normalize, linear
from arcface_torch.losses import CombinedMarginLoss, ArcFace, CosFace
from nia_age.mean_variance_loss import MeanVarianceLoss
from nia_age.main_ae import LAMBDA_1, LAMBDA_2, START_AGE, END_AGE


NUM_CLASSES = (
    len(face_age_train_dataset.class_to_idx) if face_age_train_dataset else 2154
)


class FaceFC(torch.nn.Module):
    def __init__(
        self,
        margin_loss: Callable,
        embedding_size: int,
        num_classes: int,
    ):
        super(FaceFC, self).__init__()
        self.cross_entropy = CrossEntropyLoss()
        self.embedding_size = embedding_size
        self.weight = torch.nn.Parameter(
            torch.normal(0, 0.01, (1, embedding_size))
        )
        self.margin_softmax = margin_loss
        self.num_classes = num_classes

    def forward(
        self,
        embeddings: torch.Tensor,
        labels: torch.Tensor,
    ):
        # labels
        labels.squeeze_()
        labels = labels.long()
        labels = labels.view(-1, 1)

        # embeddings
        norm_embeddings = normalize(embeddings)

        # weight
        weight = torch.nn.Parameter(
            torch.normal(0, 0.01, (self.num_classes, 512))
        ).cuda()
        norm_weight_activated = normalize(weight)
        norm_weight_activated.shape

        # logits
        logits = linear(norm_embeddings, norm_weight_activated)
        logits = logits.clamp(-1, 1)

        # softmax
        softmax = self.margin_loss(logits, labels)

        # loss
        loss = self.cross_entropy(softmax, labels.flatten())

        return loss

In [127]:
margin_loss = CombinedMarginLoss(
    64,
    *cfg.margin_list,
    cfg.interclass_filtering_threshold,
)

face_fc = FaceFC(
    margin_loss,
    512,
    NUM_CLASSES,
)

mean_variance_loss = MeanVarianceLoss(
    LAMBDA_1,
    LAMBDA_2,
    START_AGE,
    END_AGE,
)